## x.1 分类指标——准确性

我们制作的是分类任务，而在分类任务中除了loss还有一个很重要的指标——Accuracy（准确性）。我们在每一个validation过程中，增加accuracy准确性指标计算。

In [ ]:
import torch
import core

在validation step中添加accuracy的计算，并将计算结果绘制.

在下面这段代码中有意思的是这样的一段代码`Y_hat = self(*batch[:-1])`。因为我们的batch中（signal, ..., target），只有最后一个是target，所以我们通过解包取出了除了最后一个之前的数据，传入网络中产生y_hat。

In [ ]:
class Classifier(core.Module):
    def validation_step(self, batch):
        Y_hat = self(*batch[:-1])
        self.plot("loss", self.loss(Y_hat, batch[-1]), train=False)
        self.plot("acc", self.accuracy(Y_hat, batch[-1]), train=False)

使用优化器

In [ ]:
@core.add_to_class(core.Module)  #@save
def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=self.lr)

accuracy的计算，就是对于真假的计算

In [ ]:
@core.add_to_class(Classifier)
def accuracy(self, Y_hat, Y, averaged=True):
    Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
    preds = Y_hat.argmax(axis=1).type(Y.dtype)
    compare = (preds == Y.reshape(-1)).type(torch.float32)
    return compare.mean() if averaged else compare